# UE5 Multi-Agent x LLM Reasoners

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
import time
import threading
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
from dotenv import load_dotenv
from A2A.agent import A2Agent
from UE.unrealcv_a2a import UnrealCvA2A
from UE.unrealcv_basic import UnrealCV
# from ue_agent.agent import UEAgent
# from ue_agent.plan_agent import PLAgent
from llm.openai_model import UEOpenAIModel
# from ue_agent.base import ActionBuffer

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from reasoners.lm.openai_model import OpenAIModel

In [3]:
# model = OpenAIModel("gpt-4o-mini")

In [4]:
# model.generate("hello")

In [5]:
# !pip install python-dotenv

In [6]:
load_dotenv()

True

In [7]:
unrealcv_client_actor = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1280, 960))

INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [8]:
unrealcv_client_observer = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1280, 960))

INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [9]:
objects = unrealcv_client_actor.get_objects()
for obj in objects:
    if 'robot' in str(obj).lower() or 'character' in str(obj).lower():
        print(obj)

BP_SpotRobot_C_1
BP_SpotRobot_C_2
BP_Default_Character_C_2


In [10]:
objects = unrealcv_client_observer.get_objects()
for obj in objects:
    if 'robot' in str(obj).lower() or 'character' in str(obj).lower():
        print(obj)

BP_SpotRobot_C_1
BP_SpotRobot_C_2
BP_Default_Character_C_2


In [11]:
# unrealcv_client_actor.enable_controller('BP_Default_Character_C_2', True)

In [12]:
# unrealcv_client_actor.apply_action_transition('BP_Default_Character_C_2', [100, 10 , 0])

In [13]:
model = UEOpenAIModel("gpt-4o")

In [14]:
# initialize the agent
target = "Red Cube"
goal = f"""Approach the {target}
if the {target} is not in the current observation, find it by rotating the robot to looking around
then make sure the {target} is at the center of the robot's field of view
"""

des_user_prompt1 = f"""
    According to the current observations images, describe the environment in detail.
    Your description should include the following information:
    - Need to explore:Whether the {target} is in the field of view.
    - Direction:The **relative** direction of the {target} to the robot (left side means robot needs to turn left, right side means robot needs to turn right).
    - Distance:If the **relative** direction of the {target} to the robot is correct, calculate the distance to the {target} and move to it.
    - Suggestion:Suggest a high level plan on how to get to the {target}, the actions to achieve the {target} can be rotate, move, etc.(Include the specific degree description, such as slight, medium, large, as well as how to move to the object etc.)"
    just key information, plain text, no other information.
"""

des_user_prompt = f""" You are a vision-language model (VLM) agent. Based on your current visual observations, determine whether the target associated with the following goal is visible in your field of view: {goal}.
Below is an example to illustrate how you should respond:
Example Goal: “Navigate to the red mailbox.”
Example Response:
- Visibility: Yes, the red mailbox is visible.
- High-Level Plan:
  - Move forward 5 meters toward the mailbox.
  - Adjust direction slightly to the right by 15 degrees to align directly with the mailbox.
Note: The lower-level language model (LLM) does not have visual access to the environment. Therefore, you must clearly determine target visibility and provide precise, actionable instructions.
Now, using the example above, generate a similarly structured high-level plan to accomplish the current goal. """

des_system_prompt1 =  f"""
    You are a robot named <NAME> in a city.
    You have the observation of the environment, which is a list of different types of images of the environment,
    including lit, normal, depth, and object_mask images.
    Your goal is describe the environment according to your observations in detail and give the high level plan to achieve the goal:{goal} according to the description and history of actions.
    Do NOT repeat the same type of action consecutively. Such as rotate left and rotate right back again.
    Move forward when the action history is full of rotate actions.
"""

des_system_prompt =  f"""
    You are a robot named <NAME> in a city.
    You have the observation of the environment, which is a list of different images of the environment,
    Your goal is to give the high level plan to achieve the goal:{goal} according to the description and history of actions.
    Do NOT repeat the same type of action consecutively. Such as rotate left and rotate right back again.
    Move forward when the action history is full of rotate actions.
"""


In [15]:
"The suggest action for next step to achieve the {target} (rotate, move, etc.). (Include the specific degree description, such as slight, medium, large, as well as how to move to the object etc.)"

'The suggest action for next step to achieve the {target} (rotate, move, etc.). (Include the specific degree description, such as slight, medium, large, as well as how to move to the object etc.)'

In [16]:
# def get_observation(unrealcv_client, cam_id, viewmode, observation_space, description_space, agent):
#     for id in cam_id:
#         img = unrealcv_client.read_image(id, viewmode)
#         observation_space[id] = img
#         description_space[id] = agent.describe([img], system_prompt=des_user_prompt, user_prompt=des_system_prompt).text[0]
#     # with open("description.txt", "a") as f:
#     #     f.write(f"{cam_id}:\n")
#     #     f.write(f"    {description_space[id]}\n")

In [17]:
def take_action(agent: UEAgent, pagent: PLAgent):
    # print("calling action", flush=True)
    action_step = 0
    while action_step <= 50:
        camera_id = agent.camera_id
        try:
            
            observation = agent.client.read_image(camera_id, "lit", mode="file") # block
            print("finsh reading", flush=True)
            description = agent.describe([observation],system_prompt=des_user_prompt, user_prompt=des_system_prompt).text[0] # block
            # print(description, flush=True)
            print("finish_ oberved", flush=True)
            
        except Exception as e:
            print(f"error {e}", flush=True)
            print("there", flush=True)
            # continue
        steps = pagent.act(observation, description=description)
        action_step += steps
        print(f"steps{action_step}", flush=True)

In [18]:
# observation=Agent_Human.client.read_image(1, "depth")

In [19]:
# Agent_Human.describe([observation], system_prompt=des_user_prompt, user_prompt=des_system_prompt)

In [ ]:
if __name__ == "__main__":
    # action_buffer = ActionBuffer(max_size=10,unrealcv_client=unrealcv_client_actor)
    Agent_Human = A2Agent(
        model=model,
        unrealcv_client=unrealcv_client_actor,
        name="BP_Default_Character_C_2",
        goal=goal,
        temperature=1.5,
        max_history_step=5,
        camera_id=1,
        observation_viewmode='lit',
        # action_buffer=action_buffer
    )
    Agent_Dog_1 = A2Agent(
        model=model,
        unrealcv_client=unrealcv_client_actor,
        name="BP_SpotRobot_C_1",
        goal=goal,
        camera_id=3,
        temperature=1.5,
        max_history_step=5,
        observation_viewmode='lit',
        # action_buffer=action_buffer
    )
    Agent_Dog_2 = A2Agent(
        model=model,
        unrealcv_client=unrealcv_client_actor,
        name="BP_SpotRobot_C_2",
        goal=goal,
        camera_id=2,
        temperature=1.5,
        max_history_step=5,
        observation_viewmode='lit',
        # action_buffer=action_buffer
    )
    # PAgent_Human = PLAgent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_actor,
    #     name=“BP_Default_Character_C_2",
    #     goal=goal,
    #     temperature=1.5,
    #     max_history_step=5,
    #     camera_id=1,
    #     observation_viewmode=‘lit’,
    #     action_buffer=action_buffer
    #     )
    # PAgent_Dog_1 = PLAgent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_actor,
    #     name=“BP_SpotRobot_C_1”,
    #     goal=goal,
    #     camera_id=3,
    #     temperature=1.5,
    #     max_history_step=5,
    #     observation_viewmode=‘lit’,
    #     action_buffer=action_buffer
    # )
    # PAgent_Dog_2 = PLAgent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_actor,
    #     name=“BP_SpotRobot_C_2",
    #     goal=goal,
    #     camera_id=2,
    #     temperature=1.5,
    #     max_history_step=5,
    #     observation_viewmode=‘lit’,
    #     action_buffer=action_buffer
    # )
    # Agent_Observation = UEAgent(
    #     model=model,
    #     unrealcv_client=unrealcv_client_observation,
    #     name=“Observation”,
    #     goal=“Red Cube”,
    #     camera_id=1,
    # )
    Agent_Human.reset()
    Agent_Dog_1.reset()
    Agent_Dog_2.reset()
    # PAgent_Human.reset()
    # PAgent_Dog_1.reset()
    # PAgent_Dog_2.reset()
    observation_space = {}
    description_space = {}
    # observation_thread = threading.Thread(target=get_observation, args=(
    #     unrealcv_client_observation,
    #     [1, 2, 3],
    #     ‘lit’,
    #     observation_space,
    #     description_space,
    #     Agent_Observation
    # ))
    # agent_human_action_thread = threading.Thread(target=take_action, args=(Agent_Human, PAgent_Human))
    # agent_dog_1_action_thread = threading.Thread(target=take_action, args=(Agent_Dog_1, PAgent_Dog_1))
    # agent_dog_2_action_thread = threading.Thread(target=take_action, args=(Agent_Dog_2, PAgent_Dog_2))
    # print(“start observation thread”)
    # observation_thread.start()
    # time.sleep(1)
    # print(“start agent action thread”)
    # agent_human_action_thread.start()
    # agent_dog_1_action_thread.start()
    # agent_dog_2_action_thread.start()
    # time.sleep(10)
    # while action_buffer.buffer:
    #     time.sleep(5)
    #     print(“get into the loop”)
    #     action_buffer.display_actions()
    #     action_buffer.send_actions()

start agent action thread
read_image: cam_id=1, viewmode=lit, mode=file
read_image: cam_id=3, viewmode=lit, mode=file
read_image: cam_id=2, viewmode=lit, mode=file
finsh reading
finsh reading
finsh reading
GenerateOutput(text=["- Visibility: No, the Red Cube is not visible in the current observation.\n\n- High-Level Plan:\n  - Rotate left 45 degrees to scan for the Red Cube.\n  - If still not visible, rotate another 45 degrees to the left.\n  - If the Red Cube is found, adjust direction slightly to ensure it's at the center of the field of view.\n  - Move forward towards the Red Cube if visible and centered.\n  - If not yet visible after full rotation, move forward 2 meters before executing another set of rotations."], log_prob=None)
finish_ oberved
GenerateOutput(text=["- Visibility: No, the Red Cube is not currently visible in the robot's field of view.\n\n- High-Level Plan:\n  1. Rotate the robot 90 degrees to the left to scan the environment.\n  2. If the Red Cube is still not visi

Exception in thread Thread-6 (take_action):
Traceback (most recent call last):
  File "c:\Users\Shadow\miniconda3\envs\citynav\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Shadow\miniconda3\envs\citynav\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Shadow\AppData\Local\Temp\ipykernel_12632\604138139.py", line 19, in take_action
TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'


Error processing function call: empty range for randrange() (11, -10, -21)
GenerateOutput(text=["- Visibility: No, the Red Cube is not currently visible in the field of view.\n- High-Level Plan:\n  1. Rotate the robot 90 degrees to the right to scan the environment for the Red Cube.\n  2. If the Red Cube is still not visible, rotate the robot 90 degrees to the left.\n  3. If the Red Cube remains unseen, move forward 2 meters to change the location and potentially gain a new perspective.\n  4. Repeat steps 1-3 as necessary or adjust as needed based on any new visible landmarks or changes in the environment to successfully locate the Red Cube.\n  5. Once the Red Cube is visible, adjust direction to align it at the center of the robot's field of view by rotating slightly if needed."], log_prob=None)
finish_ oberved
GenerateOutput(text=['- Visibility: No, the Red Cube is not currently visible.\n- High-Level Plan:\n  - Rotate 90 degrees to the left to expand the field of view.\n  - Check fo

In [ ]:
# take_action(Agent_Human, observation_space, description_space)

In [ ]:
# image = Agent_Dog_1.client.read_image(0, "depth", mode="direct")
# image